In [1]:
from dcase_evaluator_analysis import DCASEEvaluatorAnalysis
from models.audiosep import AudioSep
# from models.one_peace_encoder import ONE_PEACE_Encoder
from models.clap_encoder import CLAP_Encoder
from models.one_peace_encoder import ONE_PEACE_Encoder

import os
from utils import parse_yaml, load_ss_model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch

from pytorch_lightning.utilities.model_summary import ModelSummary

/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/nfshomes/vla/lass-final-project/models/CLAP/open_clip/factory.py:55: FutureWarning: You a

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
# AudioSep-CLAP
config_yaml = './config/audiosep_base.yaml'
encoder_checkpoint_path = './checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'
ssnet_checkpoint_path = './checkpoint/audiosep_baseline.ckpt'


configs = parse_yaml(config_yaml)

# Load model
query_encoder = CLAP_Encoder(pretrained_path = encoder_checkpoint_path).eval()

pl_model = load_ss_model(
    configs=configs,
    checkpoint_path=ssnet_checkpoint_path,
    query_encoder=query_encoder
).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/nfshomes/vla/lass-final-project/models/CLAP/open_clip/factory.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seriali

In [ ]:
# AudioSep-CLAP param count
ModelSummary(pl_model)

  | Name          | Type         | Params | Mode 
-------------------------------------------------------
0 | ss_model      | ResUNet30    | 29.6 M | train
1 | query_encoder | CLAP_Encoder | 199 M  | eval 
-------------------------------------------------------
26.4 M    Trainable params
202 M     Non-trainable params
229 M     Total params
916.637   Total estimated model params size (MB)
156       Modules in train mode
555       Modules in eval mode

In [4]:
# AudioSep-ONE-PEACE
config_yaml = './config/audiosep_onepeace.yaml'
encoder_checkpoint_path = '/fs/nexus-scratch/vla/finetune_al_retrieval.pt'
ssnet_checkpoint_path = '/fs/nexus-scratch/vla/checkpoints/train/audiosep_onepeace,devices=1/step=140000.ckpt'


configs = parse_yaml(config_yaml)

# Load model
os.chdir('ONE-PEACE/')
query_encoder = ONE_PEACE_Encoder(pretrained_path = encoder_checkpoint_path)
query_encoder.model.model.eval()
os.chdir('..')

pl_model = load_ss_model(
    configs=configs,
    checkpoint_path=ssnet_checkpoint_path,
    query_encoder=query_encoder
).to(device)

/fs/nexus-scratch/vla/micromamba/envs/LASS/lib/python3.9/site-packages/fairseq/checkpoint_utils.py:317: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, m

In [5]:
# AudioSep-ONE-PEACE param count
ModelSummary(pl_model)

  | Name          | Type              | Params | Mode 
------------------------------------------------------------
0 | ss_model      | ResUNet30         | 39.7 M | train
1 | query_encoder | ONE_PEACE_Encoder | 0      | train
------------------------------------------------------------
36.5 M    Trainable params
3.1 M     Non-trainable params
39.7 M    Total params
158.749   Total estimated model params size (MB)
157       Modules in train mode
0         Modules in eval mode

In [ ]:
# ONE-PEACE encoder param count (in B)
sum(p.numel() for p in query_encoder.model.model.parameters()) / 1e9

2.745970785